#### passing parameters to the notebook (adding a widget)

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

#### invoke a notebook from another notebook using the %run command

In [0]:
%run "../Includes/configuration"

In [0]:
%run  "../Includes/common_functions"

In [0]:
qualifying_schema = StructType(fields=[StructField("qualifyId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), True),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("constructorId", IntegerType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("q1", StringType(), True),
                                      StructField("q2", StringType(), True),
                                      StructField("q3", StringType(), True)])

In [0]:
qualifying_df = spark.read\
    .schema(qualifying_schema)\
    .option("multiLine", True)\
    .json(f"{raw_folder_path}/{v_file_date}/qualifying")

In [0]:
qualifying_final_df = qualifying_df.withColumnRenamed("qualifyId", "qualify_id")\
                                   .withColumnRenamed("raceId", "race_id")\
                                   .withColumnRenamed("driverId", "driver_id")\
                                   .withColumnRenamed("constructorId", "constructor_id")\
                                    .withColumn("ingestion_date", current_timestamp())\
                                    .withColumn("data_source", lit(v_data_source))\
                                    .withColumn("file_date", lit(v_file_date))

In [0]:
#for race_id_list in qualifying_final_df.select("race_id").distinct().collect():
#  if (spark._jsparkSession.catalog().tableExists("f1_processed.qualifying")):
#    spark.sql(f"ALTER TABLE f1_processed.qualifying DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")
#qualifying_final_df.write.mode("append").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.qualifying")


##### incremental Load for Delta files

In [0]:
merge_condition = "tgt.qualify_id = src.qualify_id AND  tgt.race_id = src.race_id"
merge_delta_data(qualifying_final_df, "f1_processed", "qualifying", processed_folder_path, merge_condition, "race_id")

In [0]:
dbutils.notebook.exit("Success")